<a href="https://colab.research.google.com/github/hkd7148-blip/ml-portfolio/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%BF%D1%80%D0%B0%D0%B9%D1%81_%D0%BB%D0%B8%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy openpyxl sentence-transformers faiss-cpu tqdm
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.6 MB/s eta 0:00:00


In [ ]:
"""
ПОИСК АВТОЗАПЧАСТЕЙ С ИСПОЛЬЗОВАНИЕМ ВЕКТОРНОЙ БД И TELEGRAM-БОТА
Google Colab проект для учебного задания

Функционал:
1. Загрузка прайс-листов автозапчастей
2. Векторизация текста (эмбеддинги)
3. Поиск по названию + геолокации
4. Telegram-бот для поиска
"""

# ============================================================================
# ЭТАП 1: ПОДГОТОВКА СРЕДЫ
# ============================================================================

print("=" * 80)
print("ЭТАП 1: УСТАНОВКА БИБЛИОТЕК")
print("=" * 80)

# Установка необходимых библиотек
import subprocess
import sys

libraries = [
    "sentence-transformers",  # Для эмбеддингов русского текста
    "faiss-cpu",              # Векторная БД
    "pandas",                 # Работа с таблицами
    "numpy",                  # Численные операции
    "python-telegram-bot",    # Telegram бот
    "nest_asyncio",           # Для работы async в Colab
]

for lib in libraries:
    print(f"Установка {lib}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", lib])

print("✅ Все библиотеки установлены!")

# Импортируем библиотеки
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import nest_asyncio
import random
import warnings
warnings.filterwarnings('ignore')

print("\n✅ Все импорты выполнены успешно!\n")

# ============================================================================
# ЭТАП 2: СОЗДАНИЕ И ПОДГОТОВКА ДАННЫХ (ПРАЙС-ЛИСТЫ АВТОЗАПЧАСТЕЙ)
# ============================================================================

print("=" * 80)
print("ЭТАП 2: ПОДГОТОВКА ДАННЫХ - ПРАЙС-ЛИСТЫ АВТОЗАПЧАСТЕЙ")
print("=" * 80)

# Создаём датасет с прайс-листами автозапчастей
data = {
    'id': range(1, 51),
    'название_запчасти': [
        'Масло моторное Castrol 5W-30',
        'Фильтр воздушный Bosch',
        'Тормозные колодки Ferodo',
        'Аккумулятор Varta 60Ah',
        'Радиатор охлаждения',
        'Термостат двигателя',
        'Прокладка головки блока',
        'Датчик кислорода Lambda',
        'Свечи зажигания NGK',
        'Ремень газораспределения',
        'Муфта сцепления Sachs',
        'Тормозной диск Brembo',
        'Амортизатор передний Bilstein',
        'Пружина подвески',
        'Рулевая тяга',
        'Шаровая опора',
        'Втулка стабилизатора',
        'Резиновая подушка двигателя',
        'Масляный фильтр Mann',
        'Салонный фильтр Valeo',
        'Топливный фильтр Bosch',
        'Помпа водяная',
        'Генератор переменного тока',
        'Стартер двигателя',
        'Дроссельная заслонка',
        'Регулятор холостого хода',
        'Катушка зажигания Bosch',
        'Датчик температуры',
        'Датчик давления масла',
        'Датчик скорости',
        'Реле стартера',
        'Предохранитель автомобиля',
        'Трубка топливная',
        'Шланг радиатора',
        'Патрубок системы охлаждения',
        'Хомут крепления',
        'Болт крепления колеса',
        'Гайка колеса M14',
        'Прокладка бензобака',
        'Прокладка масляного картера',
        'Прокладка коробки передач',
        'Сальник коленвала',
        'Сальник распределительного вала',
        'Шланг гидроусилителя',
        'Трубка гидроусилителя',
        'Масло гидроусилителя ATF',
        'Жидкость тормозная DOT 4',
        'Антифриз G12',
        'Трансмиссионное масло 75W-90',
        'Свежий воздух салона',
    ],
    'цена': np.random.randint(100, 5000, 50),
    'город': [
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
        'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа',
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
        'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа',
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
        'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа',
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
        'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа',
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
        'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа',
    ],
    'наличие': [True] * 30 + [False] * 20,
    'описание': [
        'Синтетическое масло для двигателей до 200 тыс км',
        'Гофрированный фильтр для очистки воздуха',
        'Качественные керамические колодки',
        'Батарея 12V для всех типов авто',
        'Алюминиевый радиатор охлаждения',
        'Регулирует температуру двигателя',
        'Герметизирует головку блока цилиндров',
        'Датчик для контроля выхлопных газов',
        'Свечи для эффективного зажигания',
        'Необходим для работы двигателя',
        'Обеспечивает передачу крутящего момента',
        'Надёжное торможение автомобиля',
        'Поглощает колебания от дороги',
        'Элемент подвески автомобиля',
        'Передаёт движение рулевого колеса',
        'Шарнир в рулевой системе',
        'Гасит колебания стабилизатора',
        'Изолирует вибрацию двигателя',
        'Очищает масло двигателя',
        'Очищает воздух в салоне',
        'Очищает топливо перед впрыском',
        'Циркулирует охлаждающую жидкость',
        'Вырабатывает электроэнергию',
        'Запускает двигатель автомобиля',
        'Регулирует подачу воздуха',
        'Стабилизирует холостой ход',
        'Создаёт искру в цилиндре',
        'Мониторит температуру жидкости',
        'Контролирует давление масла',
        'Преобразует скорость вращения колёс',
        'Активирует стартер',
        'Защищает электросеть',
        'Доставляет топливо из бака',
        'Отводит охлаждающую жидкость',
        'Часть системы охлаждения',
        'Крепит трубки и патрубки',
        'Крепит колесо к ступице',
        'Крепит колёсный диск',
        'Герметизирует бензобак',
        'Герметизирует масляный картер',
        'Герметизирует коробку передач',
        'Герметизирует коленчатый вал',
        'Герметизирует распределительный вал',
        'Шланг гидравлической системы',
        'Твёрдая трубка гидросистемы',
        'Жидкость для рулевого управления',
        'Жидкость для гидравлического тормоза',
        'Охлаждающая жидкость для двигателя',
        'Масло коробки передач',
        'Фильтр для свежего воздуха',
    ]
}

# Создаём DataFrame
df = pd.DataFrame(data)

# Перемешиваем для большего разнообразия
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Создано {len(df)} записей с автозапчастями\n")

print("Первые 10 записей:")
print(df.head(10))
print(f"\nВсего колонок: {df.columns.tolist()}")
print(f"Уникальные города: {df['город'].nunique()} - {df['город'].unique().tolist()}")
print(f"Запчасти в наличии: {df['наличие'].sum()}")

# ============================================================================
# ЭТАП 3: ЗАГРУЗКА МОДЕЛИ И СОЗДАНИЕ ЭМБЕДДИНГОВ
# ============================================================================

print("\n" + "=" * 80)
print("ЭТАП 3: СОЗДАНИЕ ЭМБЕДДИНГОВ")
print("=" * 80)

# Загружаем модель для русского языка
print("\nЗагружаем модель SentenceTransformer для русского языка...")
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
print("✅ Модель загружена!")

# Создаём текстовое представление каждой запчасти
texts_for_embedding = []
for idx, row in df.iterrows():
    text = f"{row['название_запчасти']} {row['описание']} {row['город']}"
    texts_for_embedding.append(text)

print(f"\nСоздаём эмбеддинги для {len(texts_for_embedding)} запчастей...")
embeddings = model.encode(texts_for_embedding, show_progress_bar=True)
print(f"✅ Эмбеддинги созданы! Размер: {embeddings.shape}")

# ============================================================================
# ЭТАП 4: СОЗДАНИЕ ВЕКТОРНОЙ БД (FAISS)
# ============================================================================

print("\n" + "=" * 80)
print("ЭТАП 4: ПОСТРОЕНИЕ ВЕКТОРНОЙ БД")
print("=" * 80)

# Создаём индекс FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype(np.float32))

print(f"✅ Векторная БД создана!")
print(f"   Размерность вектора: {dimension}")
print(f"   Количество векторов: {index.ntotal}")

# ============================================================================
# ЭТАП 5: ФУНКЦИИ ПОИСКА
# ============================================================================

print("\n" + "=" * 80)
print("ЭТАП 5: ФУНКЦИИ ПОИСКА")
print("=" * 80)

def search_by_name(query, top_k=10):
    """Поиск по названию"""
    query_embedding = model.encode([query], show_progress_bar=False)[0]
    distances, indices = index.search(np.array([query_embedding]).astype(np.float32), top_k)

    # Правильная нормализация
    scores = 1 / (1 + distances[0])

    # Возвращаем список кортежей (int, float)
    results = []
    for idx, score in zip(indices[0], scores):
        results.append((int(idx), float(score)))

    return results


def search_by_city(city, top_k=10):
    """Поиск по городу"""
    # Получаем индексы запчастей в выбранном городе
    city_matches = df[df['город'] == city].index.tolist()

    # Создаём список результатов (индекс, score)
    results = []
    for idx in city_matches[:top_k]:
        results.append((int(idx), 1.0))

    return results

def normalize_scores(scores):
    """Нормализует оценки в диапазон [0, 1]"""
    if not scores:
        return []
    min_score = min(scores)
    max_score = max(scores)
    if max_score == min_score:
        return [1.0] * len(scores)
    return [(s - min_score) / (max_score - min_score) for s in scores]

def hybrid_search(query, city=None, top_k=5, weight_name=0.7, weight_city=0.3):
    """
    Гибридный поиск:
    - По названию (вес 0.7)
    - По городу (вес 0.3)
    """
    results = {}

    # ПОИСК ПО НАЗВАНИЮ
    print(f"🔍 Поиск по названию: '{query}'")
    name_results = search_by_name(query, top_k=20)

    for idx, score in name_results:
        idx = int(idx)
        results[idx] = {'score': float(score) * weight_name, 'source': 'name'}

    # ПОИСК ПО ГОРОДУ
    if city:
        print(f"📍 Поиск по городу: '{city}'")
        city_results = search_by_city(city, top_k=20)

        for idx, score in city_results:
            idx = int(idx)
            score = float(score)

            if idx in results:
                # Объединяем оценки
                combined_score = results[idx]['score'] + score * weight_city
                results[idx]['score'] = combined_score
            else:
                results[idx] = {'score': score * weight_city, 'source': 'city'}

    # Сортируем по итоговому score
    sorted_results = sorted(results.items(), key=lambda x: x[1]['score'], reverse=True)

    return sorted_results[:top_k]


# ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ

print("=" * 80)
print("ПРИМЕР 1: Поиск 'масло'")
print("=" * 80)

results = hybrid_search("масло", top_k=3)

for i, (idx, info) in enumerate(results, 1):
    row = df.iloc[int(idx)]
    print(f"\n{i}. {row['название_запчасти']}")
    print(f"   💰 Цена: {row['цена']} руб")
    print(f"   📍 Город: {row['город']}")
    print(f"   ⭐ Релевантность: {info['score']:.1%}")
    print(f"   📦 {'✅ Есть' if row['наличие'] else '❌ Нет'}")

print("\n" + "=" * 80)
print("ПРИМЕР 2: Гибридный поиск 'тормозные' в городе 'Москва'")
print("=" * 80)

results = hybrid_search("тормозные", city="Москва", top_k=3)

for i, (idx, info) in enumerate(results, 1):
    row = df.iloc[int(idx)]
    print(f"\n{i}. {row['название_запчасти']}")
    print(f"   💰 Цена: {row['цена']} руб")
    print(f"   📍 Город: {row['город']}")
    print(f"   ⭐ Релевантность: {info['score']:.1%}")
    print(f"   📦 {'✅ Есть' if row['наличие'] else '❌ Нет'}")

print("\n✅ ВСЕ ФУНКЦИИ РАБОТАЮТ!")

# ============================================================================
# ЭТАП 6: ТЕСТИРОВАНИЕ ПОИСКА
# ============================================================================

print("\n" + "=" * 80)
print("ЭТАП 6: ТЕСТИРОВАНИЕ ПОИСКА")
print("=" * 80)

# Тест 1: Поиск по названию
print("\n📝 Тест 1: Поиск масла моторного")
results = hybrid_search("масло моторное", top_k=5)

print("\nТоп-5 результатов:")
for i, (idx, info) in enumerate(results, 1):
    row = df.iloc[idx]
    print(f"\n{i}. {row['название_запчасти']}")
    print(f"   Цена: {row['цена']} руб")
    print(f"   Город: {row['город']}")
    print(f"   Релевантность: {info['score']:.2%}")
    print(f"   Наличие: {'✅ Есть' if row['наличие'] else '❌ Нет'}")

# Тест 2: Гибридный поиск
print("\n" + "=" * 80)
print("📝 Тест 2: Поиск тормозных колодок в Москве")
results = hybrid_search("тормозные колодки", city="Москва", top_k=5)

print("\nТоп-5 результатов:")
for i, (idx, info) in enumerate(results, 1):
    row = df.iloc[idx]
    print(f"\n{i}. {row['название_запчасти']}")
    print(f"   Цена: {row['цена']} руб")
    print(f"   Город: {row['город']}")
    print(f"   Релевантность: {info['score']:.2%}")
    print(f"   Наличие: {'✅ Есть' if row['наличие'] else '❌ Нет'}")

# ============================================================================
# ЭТАП 7: TELEGRAM БОТ
# ============================================================================

print("\n" + "=" * 80)
print("ЭТАП 7: СОЗДАНИЕ TELEGRAM БОТА")
print("=" * 80)

# Активируем asyncio для Colab
nest_asyncio.apply()

# Глобальные переменные для хранения контекста
user_context = {}

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Команда /start"""
    await update.message.reply_text(
        "🚗 Добро пожаловать в поисковик автозапчастей!\n\n"
        "Команды:\n"
        "/start - справка\n"
        "/help - помощь\n\n"
        "Как пользоваться:\n"
        "1️⃣ Отправьте название запчасти (например: 'масло моторное')\n"
        "2️⃣ Затем отправьте команду: city:Москва (опционально)\n"
        "3️⃣ Затем отправьте число: top:5 (по умолчанию 5)\n\n"
        "Пример:\n"
        "'тормозные колодки' → 'city:Санкт-Петербург' → 'top:3'"
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Команда /help"""
    await update.message.reply_text(
        "📖 Справка по командам:\n\n"
        "Поиск работает в 3 этапа:\n"
        "1. Напишите название запчасти\n"
        "2. (Опционально) Напишите 'city:ГОРОД'\n"
        "3. (Опционально) Напишите 'top:ЧИСЛО'\n\n"
        "Примеры городов: Москва, Санкт-Петербург, Новосибирск, Екатеринбург\n\n"
        "Боту нужна помощь? Отправьте /support"
    )

async def support(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Команда /support"""
    await update.message.reply_text(
        "💬 Поддержка бота:\n\n"
        "Если возникли проблемы:\n"
        "1. Проверьте правильность названия запчасти\n"
        "2. Убедитесь, что город написан с большой буквы\n"
        "3. Число результатов должно быть от 1 до 20\n\n"
        "Текущие доступные города:\n" +
        ", ".join(sorted(df['город'].unique()))
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработка сообщений"""
    user_id = update.effective_user.id
    message_text = update.message.text.strip()

    # Инициализируем контекст пользователя
    if user_id not in user_context:
        user_context[user_id] = {
            'query': None,
            'city': None,
            'top_k': 5
        }

    ctx = user_context[user_id]

    # Обработка команд
    if message_text.startswith('city:'):
        ctx['city'] = message_text.replace('city:', '').strip()
        await update.message.reply_text(f"✅ Город выбран: {ctx['city']}")
        return

    if message_text.startswith('top:'):
        try:
            ctx['top_k'] = int(message_text.replace('top:', '').strip())
            if 1 <= ctx['top_k'] <= 20:
                await update.message.reply_text(f"✅ Буду показывать топ-{ctx['top_k']} результатов")
            else:
                await update.message.reply_text("❌ Число должно быть от 1 до 20")
                ctx['top_k'] = 5
        except ValueError:
            await update.message.reply_text("❌ Введите правильное число")
        return

    # Если это название запчасти
    ctx['query'] = message_text

    # Выполняем поиск
    await update.message.reply_text("🔍 Ищу запчасти...")

    try:
        results = hybrid_search(
            ctx['query'],
            city=ctx['city'],
            top_k=ctx['top_k']
        )

        if not results:
            await update.message.reply_text("❌ Запчасти не найдены. Попробуйте другой поиск.")
            return

        # Формируем ответ
        response = f"🎯 Результаты поиска по запросу '{ctx['query']}'"
        if ctx['city']:
            response += f" в городе {ctx['city']}"
        response += ":\n\n"

        for i, (idx, info) in enumerate(results, 1):
            row = df.iloc[idx]
            response += (
                f"{i}. {row['название_запчасти']}\n"
                f"   💰 Цена: {row['цена']} руб\n"
                f"   📍 Город: {row['город']}\n"
                f"   ⭐ Релевантность: {info['score']:.1%}\n"
                f"   📦 {'✅ Есть' if row['наличие'] else '❌ Нет'}\n\n"
            )

        await update.message.reply_text(response)

    except Exception as e:
        await update.message.reply_text(f"❌ Ошибка при поиске: {str(e)}")

# Создаём приложение
print("✅ Бот создан!\n")
print("=" * 80)
print("БОТА ГОТОВ К ЗАПУСКУ!")
print("=" * 80)
print("\nЧтобы запустить бота в Google Colab, используй:")
print("app = Application.builder().token('ВАШ_TOKEN').build()")
print("app.add_handler(CommandHandler('start', start))")
print("app.add_handler(CommandHandler('help', help_command))")
print("app.add_handler(CommandHandler('support', support))")
print("app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))")
print("await app.run_polling()")

# ============================================================================
# ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ
# ============================================================================

print("\n" + "=" * 80)
print("ПРИМЕРЫ ПОИСКА")
print("=" * 80)

examples = [
    ("аккумулятор", None, 3),
    ("фильтр", "Москва", 3),
    ("масло", "Санкт-Петербург", 3),
    ("тормозные колодки", None, 3),
]

for query, city, top_k in examples:
    print(f"\n📍 Пример: Поиск '{query}'" + (f" в городе {city}" if city else ""))
    results = hybrid_search(query, city=city, top_k=top_k)

    for i, (idx, info) in enumerate(results, 1):
        row = df.iloc[idx]
        print(f"  {i}. {row['название_запчасти']} ({row['город']}) - {row['цена']} руб ⭐{info['score']:.1%}")

print("\n✅ ВСЕ ПРИМЕРЫ ВЫПОЛНЕНЫ!")

ЭТАП 1: УСТАНОВКА БИБЛИОТЕК
Установка sentence-transformers...
Установка faiss-cpu...
Установка pandas...
Установка numpy...
Установка python-telegram-bot...
Установка nest_asyncio...
✅ Все библиотеки установлены!

✅ Все импорты выполнены успешно!

ЭТАП 2: ПОДГОТОВКА ДАННЫХ - ПРАЙС-ЛИСТЫ АВТОЗАПЧАСТЕЙ
✅ Создано 50 записей с автозапчастями

Первые 10 записей:
   id             название_запчасти  цена           город  наличие  \
0  14              Пружина подвески  2094    Екатеринбург     True   
1  40   Прокладка масляного картера  2657             Уфа    False   
2  31                 Реле стартера  2094          Москва    False   
3  46      Масло гидроусилителя ATF  4092       Челябинск    False   
4  18   Резиновая подушка двигателя   635          Самара     True   
5  49  Трансмиссионное масло 75W-90  1093  Ростов-на-Дону    False   
6  27       Катушка зажигания Bosch  3857            Омск     True   
7  26      Регулятор холостого хода   149       Челябинск     True   
8  33    

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Модель загружена!

Создаём эмбеддинги для 50 запчастей...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Эмбеддинги созданы! Размер: (50, 384)

ЭТАП 4: ПОСТРОЕНИЕ ВЕКТОРНОЙ БД
✅ Векторная БД создана!
   Размерность вектора: 384
   Количество векторов: 50

ЭТАП 5: ФУНКЦИИ ПОИСКА
ПРИМЕР 1: Поиск 'масло'
🔍 Поиск по названию: 'масло'

1. Прокладка масляного картера
   💰 Цена: 2657 руб
   📍 Город: Уфа
   ⭐ Релевантность: 4.2%
   📦 ❌ Нет

2. Масляный фильтр Mann
   💰 Цена: 429 руб
   📍 Город: Ростов-на-Дону
   ⭐ Релевантность: 3.6%
   📦 ✅ Есть

3. Датчик давления масла
   💰 Цена: 100 руб
   📍 Город: Ростов-на-Дону
   ⭐ Релевантность: 3.5%
   📦 ✅ Есть

ПРИМЕР 2: Гибридный поиск 'тормозные' в городе 'Москва'
🔍 Поиск по названию: 'тормозные'
📍 Поиск по городу: 'Москва'

1. Реле стартера
   💰 Цена: 2094 руб
   📍 Город: Москва
   ⭐ Релевантность: 34.6%
   📦 ❌ Нет

2. Прокладка коробки передач
   💰 Цена: 899 руб
   📍 Город: Москва
   ⭐ Релевантность: 34.5%
   📦 ❌ Нет

3. Масло моторное Castrol 5W-30
   💰 Цена: 1486 руб
   📍 Город: Москва
   ⭐ Релевантность: 30.0%
   📦 ✅ Есть

✅ ВСЕ ФУНКЦИИ РАБОТАЮТ

In [ ]:
# Запуск Telegram бота
YOUR_TOKEN = "8276420528:AAESbDn0NiWld0WIZVJRbXMa15idh67FYyI"

app = Application.builder().token(YOUR_TOKEN).build()

# Добавляем обработчики
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("help", help_command))
app.add_handler(CommandHandler("support", support))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

print("🚀 БОТ ЗАПУЩЕН!")
print("Откройте Telegram и найдите своего бота")
print("Отправьте /start")

# Запуск
await app.run_polling()

🚀 БОТ ЗАПУЩЕН!
Откройте Telegram и найдите своего бота
Отправьте /start
🔍 Поиск по названию: 'Масло'
🔍 Поиск по названию: 'Колодки'


RuntimeError: Cannot close a running event loop